In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-05-28 13:46:22--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.59MB/s    in 0.2s    

2022-05-28 13:46:22 (5.59 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [12]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://lyricanalytics.s3.amazonaws.com/EminemWordBankTop10.csv"
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get('EminemWordBankTop10.csv'), inferSchema=True, sep=',')
# Show DataFrame
df.show()

+------------+-------------------+----------+
|    key_word|             TF-IDF|albumIndex|
+------------+-------------------+----------+
|         pee|  0.663151071321805|      1302|
|         ass|0.24276229949196215|      1302|
|        jams|0.22131638313754323|      1302|
|       nelly|0.19041670456561902|      1302|
|        move|0.18729098654674628|      1302|
|        seen|0.17909302993093332|      1302|
|     believe|0.17015806907927983|      1302|
|         way|0.16535503826559103|      1302|
|      dancer|0.16107534772073087|      1302|
|       belly| 0.1537129514539965|      1302|
|        love|0.22700268378222752|      1492|
|resuscitates|0.22588261034863172|      1492|
|    windpipe|0.22588261034863172|      1492|
|       right| 0.2254406671262595|      1492|
|     huffing|0.21567538329032365|      1492|
|       feels|0.19982533679119044|      1492|
|   suffocate|0.19822601338075793|      1492|
|         lie|0.19687247667064958|      1492|
|     alright|0.19655482111303374|

In [13]:
df2 = df.withColumnRenamed('TF-IDF','tf_idf')
df2.show()

+------------+-------------------+----------+
|    key_word|             tf_idf|albumIndex|
+------------+-------------------+----------+
|         pee|  0.663151071321805|      1302|
|         ass|0.24276229949196215|      1302|
|        jams|0.22131638313754323|      1302|
|       nelly|0.19041670456561902|      1302|
|        move|0.18729098654674628|      1302|
|        seen|0.17909302993093332|      1302|
|     believe|0.17015806907927983|      1302|
|         way|0.16535503826559103|      1302|
|      dancer|0.16107534772073087|      1302|
|       belly| 0.1537129514539965|      1302|
|        love|0.22700268378222752|      1492|
|resuscitates|0.22588261034863172|      1492|
|    windpipe|0.22588261034863172|      1492|
|       right| 0.2254406671262595|      1492|
|     huffing|0.21567538329032365|      1492|
|       feels|0.19982533679119044|      1492|
|   suffocate|0.19822601338075793|      1492|
|         lie|0.19687247667064958|      1492|
|     alright|0.19655482111303374|

In [14]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://lyrical-analysis.cxe3nmyieexj.us-east-1.rds.amazonaws.com:5432/musicalDbanalytics"
config = {"user":"<user>", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [15]:
df2.write.jdbc(url=jdbc_url, table='eminemtoptenwords', mode=mode, properties=config)